In [1]:
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

## Install new packages

You can add them into /docker/jupyter/requirements.txt or, if you are lazy, you can do it like that:

In [2]:
!pip install pandas

## Read the data

In [3]:
df = pd.read_csv('../data/winequality-red.csv')

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Upload to PostgreSQL

In [5]:
import psycopg2 as pg2

In [6]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@this_postgres')
name = 'winequality'
df.to_sql(name, engine, if_exists='replace')

## Select from PostgresSQL

In [7]:
con = pg2.connect(host='this_postgres', user='postgres',database='postgres')

In [8]:
def select(sql):
    return pd.read_sql(sql,con)

In [9]:
sql = '''select t.* from winequality t order by t.alcohol desc'''

In [10]:
df = select(sql)

In [11]:
df.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,652,15.9,0.36,0.65,7.5,0.096,22.0,71.0,0.99760,2.98,0.84,14.9,5
1,467,8.8,0.46,0.45,2.6,0.065,7.0,18.0,0.99470,3.32,0.79,14.0,6
2,1270,5.0,0.38,0.01,1.6,0.048,26.0,60.0,0.99084,3.70,0.75,14.0,6
3,142,5.2,0.34,0.00,1.8,0.050,27.0,63.0,0.99160,3.68,0.79,14.0,6
4,1269,5.5,0.49,0.03,1.8,0.044,28.0,87.0,0.99080,3.50,0.82,14.0,8


Your data will be stored in PostgreSQL volume. Even if your delete containers and then recreate them, all data will be in place.

## Calendar table

https://medium.com/@duffn/creating-a-date-dimension-table-in-postgresql-af3f8e2941ac

There is a table with all possible dates. Use it to create periodical reports, e.g. monthly report.

In [12]:
sql = '''select * from d_date t limit 10'''

In [13]:
select(sql)

,date_dim_id,date_actual,epoch,day_suffix,day_name,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,week_of_year,week_of_year_iso,month_actual,month_name,month_name_abbreviated,quarter_actual,quarter_name,year_actual,first_day_of_week,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year,mmyyyy,mmddyyyy,weekend_indr
0,19700101,1970-01-01,0,1st,Thursday,4,1,1,1,1,1,1970-W01-4,1,January,Jan,1,First,1970,1969-12-29,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01011970,False
1,19700102,1970-01-02,86400,2nd,Friday,5,2,2,2,1,1,1970-W01-5,1,January,Jan,1,First,1970,1969-12-29,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01021970,False
2,19700103,1970-01-03,172800,3rd,Saturday,6,3,3,3,1,1,1970-W01-6,1,January,Jan,1,First,1970,1969-12-29,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01031970,True
3,19700104,1970-01-04,259200,4th,Sunday,7,4,4,4,1,1,1970-W01-7,1,January,Jan,1,First,1970,1969-12-29,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01041970,True
4,19700105,1970-01-05,345600,5th,Monday,1,5,5,5,1,2,1970-W02-1,1,January,Jan,1,First,1970,1970-01-05,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01051970,False
5,19700106,1970-01-06,432000,6th,Tuesday,2,6,6,6,1,2,1970-W02-2,1,January,Jan,1,First,1970,1970-01-05,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01061970,False
6,19700107,1970-01-07,518400,7th,Wednesday,3,7,7,7,1,2,1970-W02-3,1,January,Jan,1,First,1970,1970-01-05,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01071970,False
7,19700108,1970-01-08,604800,8th,Thursday,4,8,8,8,2,2,1970-W02-4,1,January,Jan,1,First,1970,1970-01-05,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01081970,False
8,19700109,1970-01-09,691200,9th,Friday,5,9,9,9,2,2,1970-W02-5,1,January,Jan,1,First,1970,1970-01-05,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01091970,False
9,19700110,1970-01-10,777600,10th,Saturday,6,10,10,10,2,2,1970-W02-6,1,January,Jan,1,First,1970,1970-01-05,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,011970,01101970,True


## Create your tables

In [14]:
cur = con.cursor()

In [15]:
sql = '''
drop table if exists hello_table;
create table hello_table as 
select 'hello!!!!' as col
'''

In [16]:
cur.execute(sql)

In [17]:
sql = '''select * from hello_table t'''

In [18]:
select(sql)

,col
0,hello!!!!


## Interactive Development

A major goal of this project to framework is to facilitate a new style of software development called interactive development. The interactive development of modules is as follows.

1. Use Jupyter to write code interactively in a notebook.
2. When a block of code gets too large or needs to be repeated, abstract this code into a function in Jupyter.
3. Test the performance of this new function in Jupyter.
4. Move this function to a module in your library of code.
5. Import the code for use as needed.

In [19]:
from os import chdir
chdir('/home/jovyan')
import lib.hello as hello

In [20]:
hello.hello('Jupyter')

Hello Jupyter!
